In [ ]:
#from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import integrate as integrate
from matplotlib import gridspec
import scipy.signal
import healpy as hp
#import Corrfunc
from numba import jit

In [ ]:
plt.rc("text", usetex=True)
plt.rc("font", size=20)
plt.rcParams["axes.linewidth"]  = 1.5
plt.rcParams["xtick.major.size"]  = 8
plt.rcParams["xtick.minor.size"]  = 3
plt.rcParams["ytick.major.size"]  = 8
plt.rcParams["ytick.minor.size"]  = 3
plt.rcParams["xtick.direction"]  = "in"
plt.rcParams["ytick.direction"]  = "in"
plt.rcParams["legend.frameon"] = 'False'


In [ ]:
val = np.random.randint(0, 50)
print(val)

In [ ]:
@jit(nopython=True)
def poisson_sampling(hpmap,n_sample):
    count = 0
    high = len(hpmap)
    count_map = np.zeros(high)
    print(hpmap.min(), hpmap.max())
    while count<n_sample:
        pix = np.random.randint(0,len(hpmap))
        pix_count = np.random.poisson(1.0+hpmap[pix])
        count_map[pix] += pix_count
        count += pix_count
        
    return count_map

def poisson_sampling_v2(hpmap, n_sample):
    norm_map = 1. + hpmap
    norm_map *= (n_sample / norm_map.sum())
    count_map = np.random.poisson(norm_map.clip(0, np.nan))
    return count_map

# generate a density map using cl
def density_cl(cl, nside, randomSeed):
    np.random.seed(randomSeed)
    alm = hp.sphtfunc.synalm(cl,lmax = 3*nside -1)
    density = hp.sphtfunc.alm2map(alm, nside,  verbose=False)
    return density

def overdensityMap(countsMap):
    mean = np.mean(countsMap)
    return ( countsMap - mean ) / mean


def w_cross_IsotripicMaps_std(cl_isotropic, nside, N_event, overdensityMap_g, N_re=100):
    # N_re = number of realizations
    w_cross_random = np.zeros((N_re, 3*nside))
    # Generate an isotropic neutrino sample 
    for i in range (N_re):
        density_random = density_cl(cl_isotropic, nside, i+50)
        events_map_random = poisson_sampling(density_random,N_event)
        overdensity_random = overdensityMap(events_map_random)
        w_cross_random[i] = hp.sphtfunc.anafast(overdensityMap_g, overdensity_random)
    return np.std(w_cross_random, axis=0)
    
def w_cross_astrophysical_atm(cl_isotropic, nside, Natm_Nnu, eventMap_nu, overdensityMap_g, N_re=100):
    density_isotropic = density_cl(cl_isotropic, nside, 5098)
    events_map_isotropic = poisson_sampling(density_isotropic,Natm_Nnu * N_nu)
    events_map_combined = eventMap_nu + events_map_isotropic
    overdensityMap_combined = overdensityMap(events_map_combined)
    #cl_combined = hp.sphtfunc.anafast(overdensityMap_combined)
    w_cross_combined = hp.sphtfunc.anafast(overdensityMap_g,overdensityMap_combined)
    # comparing with a pure background
    w_std = w_cross_IsotripicMaps_std(cl_isotropic, nside, N_nu * (Natm_Nnu + 1), overdensityMap_g, N_re)
    chi_square = np.sum(w_cross_combined[1:]**2 / w_std[1:]**2)
    return significance(chi_square, len(w_cross_combined[1:]))


    
    
from scipy.stats import chi2, norm, distributions
def significance(chi_square, dof):
    p_value = distributions.chi2.sf(chi_square, dof-1)
    significance_twoTailNorm = norm.ppf(1. - p_value / 2)
    return significance_twoTailNorm 



## Create a power spectrum

In [ ]:
NSIDE = 128
l = np.linspace(1, 500, 500)
cl = 0.005/l**1.5 # Power spectrum of the "normal" galaxies
cl_2 = 0.005*1.3/l**1.5 #Power spectrum from which the galaxies hosting high energy neutrino events will be drawn

In [ ]:
plt.loglog(l,cl)
plt.loglog(l,cl_2)

## Create a realization of each power spectrum. Note that the realization seed should be the same

In [ ]:
density_g = density_cl(cl, NSIDE, 42)
density_nu = density_cl(cl_2, NSIDE, 42)
hp.mollview(density_g)
hp.mollview(density_nu)

## Poisson sample each map. Map 1 gives the galaxy density map. Map 2 gives the neutrino events map

In [ ]:
(density_g.min(), density_g.max())

In [ ]:
## Poisson sample each map. Map 1 gives the galaxy density map. Map 2 gives the neutrino events mapevents_map_g = poisson_sampling(density_g, 2000000)
N_g = 2000000
N_nu = 2000
events_map_g = poisson_sampling(density_g, N_g)
events_map_nu = poisson_sampling(density_nu, N_nu)
hp.mollview(events_map_g)
hp.mollview(events_map_nu)

In [ ]:
print(events_map_g.sum(), events_map_nu.sum())

## Convert density maps to overdensities

In [ ]:
overdensityMap_g = overdensityMap(events_map_g)
overdensityMap_nu = overdensityMap(events_map_nu)

l_cl = np.arange(1,3*NSIDE +1)
cl_g = hp.sphtfunc.anafast(overdensityMap_g)
cl_nu = hp.sphtfunc.anafast(overdensityMap_nu)
w_cross = hp.sphtfunc.anafast(overdensityMap_g, overdensityMap_nu)


In [ ]:
plt.plot(l,cl)
plt.plot(l_cl[1:],cl_g[1:], label='g')
plt.plot(l_cl[1:],cl_nu[1:],label='nu')
plt.plot(l_cl[1:],np.abs(w_cross[1:]), label='cross')

plt.xscale('log')
plt.yscale('log')
plt.legend()

# Generate an isotropic map

In [ ]:
cl_isotropic = l*0.
density_isotropic = density_cl(cl_isotropic, NSIDE, 202)
hp.mollview(density_isotropic)
events_map_isotropic = poisson_sampling(density_isotropic,N_nu)
hp.mollview(events_map_isotropic)
# the cross correlation level of galaxy samples with an isotropic background
overdensityMap_isotropic = overdensityMap(events_map_isotropic)
w_cross_isotropic = hp.sphtfunc.anafast(overdensityMap_g, overdensityMap_isotropic)


# Calculate uncertainites of the cross-correlation function 

In [ ]:
w_std = w_cross_IsotripicMaps_std(cl_isotropic, NSIDE, N_nu, overdensityMap_g)

In [ ]:
plt.plot(l_cl[1:], w_cross[1:])

plt.errorbar(l_cl[1:], l_cl[1:] * 0., yerr=w_std[1:])
plt.ylim(-1e-3, 1e-3)
plt.plot(l_cl, l_cl * 0, '--')


In [ ]:
chi_square = np.sum(w_cross[1:]**2 / w_std[1:]**2)
print(significance(chi_square, len(w_cross[1:])))


# Signal with an isotropic background

In [ ]:
Natm_Nnu = 100
events_map_isotropic2 = poisson_sampling(density_isotropic,Natm_Nnu * N_nu)
events_map_combined = events_map_nu + events_map_isotropic2
#hp.mollview(events_map_combined)

In [ ]:
overdensityMap_combined = overdensityMap(events_map_combined)
#cl_combined = hp.sphtfunc.anafast(overdensityMap_combined)
w_cross_combined = hp.sphtfunc.anafast(overdensityMap_g,overdensityMap_combined)
# comparing with a pure background
w_std2 = w_cross_IsotripicMaps_std(cl_isotropic, NSIDE, N_nu * (Natm_Nnu + 1), overdensityMap_g)
chi_square2 = np.sum(w_cross_combined[1:]**2 / w_std2[1:]**2)
print(significance(chi_square2, len(w_cross_combined[1:])))


In [ ]:
print(w_cross_astrophysical_atm(cl_isotropic, NSIDE, Natm_Nnu, events_map_nu, overdensityMap_g))



plt.plot(l_cl[1:], w_cross_combined[1:])

plt.errorbar(l_cl[1:], l_cl[1:] * 0., yerr=w_std2[1:])
plt.ylim(-1e-4, 1e-4)
plt.plot(l_cl, l_cl * 0, '--')



## Realistic Galaxy Power Spectrum

In [ ]:
cl_galaxy_file = np.loadtxt('Cl_ggRM.dat')
cl_galaxy = cl_galaxy_file[:500]


In [ ]:
plt.loglog(l,cl)
plt.loglog(l, cl_galaxy)

In [ ]:
density_g = density_cl(cl_galaxy, NSIDE, 42)
density_g = np.exp(density_g) - 1.0
density_nu = density_cl(cl_galaxy * 1.3, NSIDE, 42)
density_nu = np.exp(density_nu) - 1.0
hp.mollview(density_g)
hp.mollview(density_nu)

In [ ]:
N_g = 2000000
N_nu = 2000
events_map_g = poisson_sampling(density_g, N_g)
events_map_nu = poisson_sampling(density_nu, N_nu)
hp.mollview(events_map_g)
hp.mollview(events_map_nu)

In [ ]:
overdensityMap_g = overdensityMap(events_map_g)
overdensityMap_nu = overdensityMap(events_map_nu)

l_cl = np.arange(1,3*NSIDE +1)
cl_g = hp.sphtfunc.anafast(overdensityMap_g)
cl_nu = hp.sphtfunc.anafast(overdensityMap_nu)
w_cross = hp.sphtfunc.anafast(overdensityMap_g, overdensityMap_nu)

In [ ]:
plt.plot(l,cl_galaxy)
plt.plot(l_cl[1:],cl_g[1:], label='g')
plt.plot(l_cl[1:],cl_nu[1:],label='nu')
plt.plot(l_cl[1:],np.abs(w_cross[1:]), label='cross')

plt.xscale('log')
plt.yscale('log')
plt.legend()

In [ ]:
w_std = w_cross_IsotripicMaps_std(cl_isotropic, NSIDE, N_nu, overdensityMap_g)
chi_square = np.sum(w_cross[1:]**2 / w_std[1:]**2)
print(significance(chi_square, len(w_cross[1:])))